In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_data_no_embed.pkl"
full_data=loadpkl(path)

In [26]:
train_data={}
for k in full_data:
    data={}
    data['paperID']=k

    if full_data[k]['abstract'] is not None:
        data['abstract']=full_data[k]['abstract']
    else:
        data['abstract']=""

    if full_data[k]['title'] is not None:
        data['title']=full_data[k]['title']
    else:
        data['title']=""

    data['authors']=full_data[k]['authors']
    train_data[k]=data
path=f"/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_data_abs_title_author_data.pkl"
with open(path,'wb') as f:
    pickle.dump(train_data,f)

In [27]:
print(len(train_data)) 
for k,v in train_data.items():
    print(k)
    print(v)
    break

2037520
5fd27a206360395fe1a2c883d3f3e372e8c31209
{'paperID': '5fd27a206360395fe1a2c883d3f3e372e8c31209', 'abstract': '', 'title': 'Novel Algorithm to Calculate Hypervolume Indicator of Pareto Approximation Set', 'authors': [{'authorId': '2149536821', 'name': 'Qing Yang'}, {'authorId': '33713842', 'name': 'Shengchao Ding'}]}


In [7]:
for k,v in full_data.items():
    print(v.keys())
    print(v['authors'])
    print(v['title'])
    print(v['abstract'])
    break

dict_keys(['paperId', 'externalIds', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'fieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'citationStyles', 'authors', 'references'])
[{'authorId': '2149536821', 'name': 'Qing Yang'}, {'authorId': '33713842', 'name': 'Shengchao Ding'}]
Novel Algorithm to Calculate Hypervolume Indicator of Pareto Approximation Set
None


In [8]:
train=loadpkl('/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/train.pkl')

In [9]:
for k in train:
    print(k)
    break

acb6fd4058b3c6ce491d5cde499d7733909bc8a9


In [24]:
for mode in ['train','dev','test']:
    path=f"/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/{mode}_filtered.pkl"
    train=loadpkl(path)
    train_data={}
    for k in train:
        data={}
        data['paperID']=k
        
        if full_data[k]['abstract'] is not None:
            data['abstract']=full_data[k]['abstract']
        else:
            data['abstract']=""
            
        if full_data[k]['title'] is not None:
            data['title']=full_data[k]['title']
        else:
            data['title']=""
            
        data['authors']=full_data[k]['authors']
        train_data[k]=data
    path=f"/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/{mode}_filtered_abs_title_author_data.pkl"
    with open(path,'wb') as f:
        pickle.dump(train_data,f)

In [7]:
data=loadpkl('/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/test_filtered_abs_title_author_data.pkl')

In [8]:
print(len(data))
for k,v in data.items():
    x=v
    print(v)
    break

968
{'paperID': '0599083e7f6c2574a24f63a7510e39598db46623', 'abstract': 'We introduce Dreamento (Dream engineering toolbox), an open-source Python package for dream engineering using sleep electroencephalography (EEG) wearables. Dreamento main functions are (1) real-time recording, monitoring, analysis, and sensory stimulation, and (2) ofﬂine post-processing of the resulting data, both in a graphical user interface (GUI). In real-time, Dreamento is capable of (1) data recording, visualization, and navigation, (2) power-spectrum analysis, (3) automatic sleep scoring, (4) sensory stimulation (visual, auditory, tactile), (5) establishing text-to-speech communication, and (6) managing annotations of automatic and manual events. The ofﬂine functions aid in post-processing the acquired data with features to reformat the wearable data and integrate it with non-wearable recorded modalities such as electromyography (EMG). While Dreamento was primarily developed for (lucid) dreaming studies, its

In [15]:
sent1=' and '.join([item['name'] for item in x['authors']])
print(sent1)
sent2=x['title']+'.'+'\n'+x['abstract']
print(sent2)

Mahdad Jafarzadeh Esfahani and Amir Hossein Daraie and F. Weber and M. Dresler
Dreamento: an open-source dream engineering toolbox for sleep EEG wearables.
We introduce Dreamento (Dream engineering toolbox), an open-source Python package for dream engineering using sleep electroencephalography (EEG) wearables. Dreamento main functions are (1) real-time recording, monitoring, analysis, and sensory stimulation, and (2) ofﬂine post-processing of the resulting data, both in a graphical user interface (GUI). In real-time, Dreamento is capable of (1) data recording, visualization, and navigation, (2) power-spectrum analysis, (3) automatic sleep scoring, (4) sensory stimulation (visual, auditory, tactile), (5) establishing text-to-speech communication, and (6) managing annotations of automatic and manual events. The ofﬂine functions aid in post-processing the acquired data with features to reformat the wearable data and integrate it with non-wearable recorded modalities such as electromyograp

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [17]:
_input=tokenizer.encode_plus(
            sent1,
            sent2,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
        )

In [18]:
print(_input)

{'input_ids': [101, 5003, 14945, 4215, 14855, 14971, 4143, 25383, 9686, 28975, 2072, 1998, 18904, 7570, 11393, 2378, 18243, 4886, 2063, 1998, 1042, 1012, 13351, 1998, 1049, 1012, 2852, 2229, 3917, 102, 3959, 4765, 2080, 1024, 2019, 2330, 1011, 3120, 3959, 3330, 6994, 8758, 2005, 3637, 25212, 2290, 4929, 3085, 2015, 1012, 2057, 8970, 3959, 4765, 2080, 1006, 3959, 3330, 6994, 8758, 1007, 1010, 2019, 2330, 1011, 3120, 18750, 7427, 2005, 3959, 3330, 2478, 3637, 16175, 10127, 21890, 21197, 26228, 1006, 25212, 2290, 1007, 4929, 3085, 2015, 1012, 3959, 4765, 2080, 2364, 4972, 2024, 1006, 1015, 1007, 2613, 1011, 2051, 3405, 1010, 8822, 1010, 4106, 1010, 1998, 16792, 20858, 1010, 1998, 1006, 1016, 1007, 1997, 30511, 3170, 2695, 1011, 6364, 1997, 1996, 4525, 2951, 1010, 2119, 1999, 1037, 20477, 5310, 8278, 1006, 26458, 1007, 1012, 1999, 2613, 1011, 2051, 1010, 3959, 4765, 2080, 2003, 5214, 1997, 1006, 1015, 1007, 2951, 3405, 1010, 5107, 3989, 1010, 1998, 9163, 1010, 1006, 1016, 1007, 2373, 1011,

In [19]:
print(len(_input['input_ids']))

512
